In [132]:
import pandas as pd
from datetime import datetime

In [133]:
customers = pd.read_csv('data/customers.csv')
flowers = pd.read_csv('data/flowers.csv')
orders = pd.read_csv('data/orders.csv')
shipments = pd.read_csv('data/shipments.csv')

# Average damage per shipment

In [134]:
shipments_flowers = pd.merge(shipments, flowers, how='inner', on='flower_id')

In [135]:
shipments_flowers.shipment_date = pd.to_datetime(shipments_flowers.shipment_date, format='%Y/%m/%d %I:%M %p')

In [136]:
shipments_flowers['days_in_storage'] = (datetime(2024, 2, 11) - shipments_flowers.shipment_date).dt.days

In [137]:
def average_damage(df):
    '''receive a dataframe with two columns called 
    damage_function and days_in_storage 
    returns the average damage of the shipment which is a
    percentaje
    '''
    df['average_damage'] = (1 / df.damage_function) * df.days_in_storage

    for i in df.index:
        df.loc[i, 'average_damage'] = min(df.loc[i, 'average_damage'], 100) / 100
        
    return df

In [138]:
shipments_flowers = average_damage(shipments_flowers)

In [139]:
shipments_flowers['quantity_no_damage'] = shipments_flowers['quantity'] * (1 - shipments_flowers['average_damage'])

In [140]:
shipments_flowers.head()

,shipment_id,shipment_date,flower_id,quantity,flower_name,damage_function,cost_cad,days_in_storage,average_damage,quantity_no_damage
0,0,2024-01-08 17:35:00,4,738,bouquet_rose_orange,1.0,10,33,0.330000,494.460000
1,1,2024-01-09 05:28:00,5,1673,pom,1.5,7,32,0.213333,1316.093333
2,2,2024-01-11 16:56:00,10,1330,spiders,2.0,9,30,0.150000,1130.500000
3,3,2024-01-19 04:43:00,0,1950,alstromelia,3.0,5,22,0.073333,1807.000000
4,4,2024-01-19 16:41:00,4,1853,bouquet_rose_orange,1.0,10,22,0.220000,1445.340000


In [141]:
flowers_type_damage = shipments_flowers[['flower_id', 'flower_name', 'quantity_no_damage', 'quantity']
].groupby(['flower_id', 'flower_name']).sum().reset_index()

In [142]:
flowers_type_damage.head()

,flower_id,flower_name,quantity_no_damage,quantity
0,0,alstromelia,18638.466667,20489
1,1,bouquet_rose_white,18143.600000,21545
2,2,bouquet_rose_red,28080.330000,34966
3,3,bouquet_rose_yellow,21913.780000,28221
4,4,bouquet_rose_orange,23199.150000,29104


# Orders

In [143]:
orders.order_date = pd.to_datetime(orders.order_date, format='%Y/%m/%d %I:%M %p')

In [144]:
orders_flowers = pd.merge(orders, flowers, how='inner', on='flower_id')

In [145]:
orders_flowers = orders_flowers.drop(columns = 'damage_function')

In [146]:
orders_flowers.head()

,order_id,customer_id,order_date,flower_id,quantity,flower_name,cost_cad
0,0,335,2024-03-19 22:49:00,7,26,hydrangea,10
1,1,35,2024-04-16 05:47:00,5,24,pom,7
2,2,295,2024-03-07 00:25:00,5,20,pom,7
3,3,421,2024-03-03 03:42:00,3,22,bouquet_rose_yellow,10
4,4,156,2024-04-11 14:16:00,4,20,bouquet_rose_orange,10


# Money loss

In [147]:
shipments_flowers['money_lost_damage'] = (shipments_flowers['quantity'] - shipments_flowers['quantity_no_damage']
                                         ) * shipments_flowers['cost_cad']

In [148]:
shipments_flowers.money_lost_damage.sum()

380564.31166666665

In [149]:
shipments_flowers['inventory_gross_value'] = shipments_flowers['quantity'] * shipments_flowers['cost_cad']

In [150]:
shipments_flowers.inventory_gross_value.sum()

2392058

# Create views

In [151]:
shipments_flowers.to_csv('views/shipments_flowers.csv', index=False)
flowers_type_damage.to_csv('views/flowers_type_damage.csv', index=False)
orders_flowers.to_csv('views/orders_flowers.csv', index=False)